In [2]:
import sqlite3 as db
import pandas as pd
import statsmodels.api as sm
from datetime import datetime
database = "/home/sachin/Downloads/RWO_0004_Ventilatoren_00.sqlite"

con = db.connect(database)
df = pd.read_sql_query(f"SELECT time, value FROM Value WHERE sensor_id=2",
                       con)
df["time"] = df["time"].apply(lambda utc: datetime.fromtimestamp(int(utc / 1000)))
df.drop_duplicates(subset="time", keep="first", inplace=True)
df.index = df['time']
df.drop('time', axis=1, inplace=True)
df = df.reindex(pd.date_range(min(df.index),
                              max(df.index),
                              freq='S'))
df = df.interpolate().fillna(method='bfill')
con.close()

In [3]:
print(df)


                        value
2021-04-23 02:00:27  0.468526
2021-04-23 02:00:28  0.468535
2021-04-23 02:00:29  0.468544
2021-04-23 02:00:30  0.468553
2021-04-23 02:00:31  0.468562
...                       ...
2022-05-16 14:34:29 -3.749998
2022-05-16 14:34:30 -3.749999
2022-05-16 14:34:31 -3.750000
2022-05-16 14:34:32 -3.750000
2022-05-16 14:34:33 -3.750000

[33568447 rows x 1 columns]


In [ ]:
result = sm.tsa.stattools.adfuller(df["value"])
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

In [ ]:
if result[1] < 0.05 and result[0] < result[4]['5%']:
    print("Stationary")
else:
    print("Non Stationary")

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
res=seasonal_decompose(df["value"], model='additive',period=1)
def plotseasonal(res):
    res.observed.plot(ax=axes[0], legend=False)
    axes[0].set_ylabel('Observed')
    res.trend.plot(ax=axes[1], legend=False)
    axes[1].set_ylabel('Trend')
    res.seasonal.plot(ax=axes[2], legend=False)
    axes[2].set_ylabel('Seasonal')
    res.resid.plot(ax=axes[3], legend=False)
    axes[3].set_ylabel('Residual')

fig, axes = plt.subplots(nrows=4, sharex=True)
fig.suptitle('Seasonal Decomposition of Vibration Values for a week')
plotseasonal(res)
plotseasonal(res)
plotseasonal(res)

plt.tight_layout(pad=1.08, rect=[0, 0.03, 1, 0.95])

# result.plot()
plt.savefig('decomposition.jpg', format='jpg', dpi=1200)

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as smt

def tsplot(y, lags=None, figsize=(12, 7)):
    """
        Plot time series, its ACF and PACF, calculate Dickey–Fuller test

        y - timeseries
        lags - how many lags to include in ACF, PACF calculation
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    fig = plt.figure(figsize=figsize)
    layout = (2, 1)
    #ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
    acf_ax = plt.subplot2grid(layout, (0, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 0))

    #y.plot(ax=ts_ax)
    #p_value = sm.tsa.stattools.adfuller(y)[1]
    #ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
    smt.plot_acf(y, lags=lags, ax=acf_ax)
    smt.plot_pacf(y, lags=lags, ax=pacf_ax)
    plt.tight_layout()

tsplot(df['value'])

In [ ]:
df = df.iloc[:10000]
test_len = int(len(df) * 0.2)
train, test = df.iloc[:-test_len], df.iloc[-test_len:]

# from sktime.forecasting.arima import AutoARIMA
#
# forecaster = AutoARIMA(start_p=2, max_p=5, suppress_warnings=True)
# train.index = train.index.astype(int)
# forecaster.fit(train)
# print(forecaster.summary())


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# fh = np.arange(test_len) + 1
# forecast = forecaster.predict(fh=fh, alpha=0.05)
# print(mean_squared_error(test, forecast))
from statsmodels.tsa.statespace.sarimax import SARIMAX



# from sktime.forecasting.arima import ARIMA
# model = ARIMA(train, (4,0,0)).fit()
# pred = model.predict(start=len(train))

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
# from statsmodels.tsa.arima.model import  import ARIMA
order = (4, 1, 2)
seasonal_order = (1, 1, 1, 60)
# model = SARIMAX(endog=train, order=order)
model = SARIMAX(endog=train, order=order, seasonal_order=seasonal_order)
results = model.fit()

In [ ]:
print(len(test))
yhat = results.forecast(steps = len(test))
print(yhat)
lm = pd.DataFrame(index=test.index)
lm['pred'] = yhat.values
lm['y'] = test['value']
print(lm)
plt.plot(lm)
plt.show()
